<a href="https://colab.research.google.com/github/lizzifer991/Big-Data_Challenge/blob/main/Level-1/Lvl1_Books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Dependencies
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 256 kB in 12s (22.1 kB/s)
Reading

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-13 06:21:38--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.20MB/s    in 0.2s    

2022-09-13 06:21:39 (4.20 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Books").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)

In [5]:
book_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_01.tsv.gz"), sep="\t", header=True)

#Show DataFrame
book_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [6]:
#Count Records
book_df.count()

6106719

In [7]:
#DropNA's
book_df = book_df.dropna()

In [8]:
#DropDuplicates
book_df = book_df.dropDuplicates()

In [9]:
#Count records
book_df.count()

6105983

In [10]:
#Create a Review_df
Review_df= book_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
#DropDuplicates
Review_df= Review_df.dropDuplicates()
#DropNA's
Review_df= Review_df.dropna()
#Show Review_df
Review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100KP9PY8KY9Z|   51671847|1440208018|     944515458| 2011-01-25|
|R100UP9DK05S1V|   51147790|1847199364|     612608608| 2011-01-26|
|R100VOIR4CMMWN|   26798794|1585745308|     240577394| 2012-03-13|
|R100ZIBB5BENPU|   36680692|1432768824|      52920125| 2011-04-04|
| R1012O05VE5YU|   11936223|1601370393|     575056867| 2008-04-20|
|R1014BD065DL3T|   15962088|1401918697|     967458723| 2008-08-23|
|R1017RPDK4SOBA|   29969814|0786444266|     822704558| 2011-06-02|
|R101BBTWRCXZAZ|   34172149|193328577X|     668924177| 2008-04-10|
|R101FHR47R4IQO|   50257321|0140567585|     282954431| 2009-07-31|
|R101GA76XNOLCP|   43219793|0470607742|     272450781| 2012-04-04|
|R101JF8RCF4Z0R|   21450606|1413759793|     395153107| 2005-12-21|
|R101K58OQT9RM1|   14638404|0767925203|     912028162| 2007-09

In [11]:
Review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- review_date: string (nullable = true)



In [12]:
#Create a Product_df
Product_df= book_df.select(["product_id","product_title"])
#DropDuplicates
Product_df= Product_df.dropDuplicates()
#DropNA's
Product_df= Product_df.dropna()
#Show Review_df
Product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1419701630|Chuck Close: Face...|
|0451216954|Dark Lover (Black...|
|0552142395|My Feudal Lord: A...|
|1931514941|   Love Hina, Vol. 1|
|0615462219|The Great Pain De...|
|0805087605|What's Inside You...|
|1439197334|Far from Here: A ...|
|1413748260|       Plasma Dreams|
|1613469845|           Bolt Clan|
|B005IUSVS0|       If I Were You|
|0486451550|Architectura Nava...|
|0312426267|Golden Boy: Memor...|
|0425247414|Murder on Fifth A...|
|0375722203|Random House Webs...|
|1615390588|Colodin Project, ...|
|047023847X|HTML, XHTML and C...|
|1892785390|Continuing Care S...|
|1467996149|Far Away in the S...|
|0307886948|Jeannie Out of th...|
|159038668X|The Kingdom and t...|
+----------+--------------------+
only showing top 20 rows



In [13]:
Product_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [14]:
#Create a Customers_df
Customers_df= book_df.groupby("customer_id")\
              .agg({"customer_id": "count"})\
              .withColumnRenamed("count(customer_id)", "customer_count")
#DropDuplicates
Customers_df= Customers_df.dropDuplicates()
#DropNA's
Customers_df= Customers_df.dropna()
#Show Review_df
Customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   36500827|            82|
|   33108857|             3|
|   36862491|             1|
|   50055266|             7|
|   46238124|             1|
|   14368851|            16|
|   10381323|             1|
|   52175489|             7|
|   12355043|             2|
|   49762922|             1|
|   52862315|             2|
|   27251832|           348|
|   12457227|           102|
|   44153355|           549|
|   14463307|             3|
|   39101891|             1|
|   30409893|             3|
|   34542723|             1|
|   52304218|            30|
|   14818969|           208|
+-----------+--------------+
only showing top 20 rows



In [15]:
Customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_count: long (nullable = false)



In [16]:
#Count Review Records
Review_df.count()

6105983

In [17]:
#Count Product Records
Product_df.count()

1502163

In [18]:
#Count Customer Records
Customers_df.count()

2730605

In [19]:
# Create vine_table
vine_table_df = book_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10067I2MPT3S9|          5|           18|         23|   N|
|R1007R0YE5NDHU|          5|            2|          2|   N|
| R100JBOXCODNI|          5|           12|         15|   N|
|R100MH205G3DF9|          4|            0|          3|   N|
|R100YGSB2I3P2M|          3|            0|          1|   Y|
|R100ZZRLAKS30W|          4|            1|          1|   N|
|R10105K7MP8S9M|          3|            7|          7|   N|
|R1010SEABEJ235|          5|            8|          8|   N|
|R1012CPNI64JSP|          5|           11|         11|   N|
|R1015ALNED2LVI|          5|            4|          4|   N|
|R101HCCIZR7L0O|          5|            1|          1|   N|
|R101YXFD3NORUE|          5|           16|         24|   N|
|R1027EAE93VBMB|          2|            1|          3|   N|
|R102AQRLGHH1MG|          5|            

In [20]:
#Configure for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cvwouuigjbgo.us-east-2.rds.amazonaws.com:5432/amazon_reviews_us_Books_v1_01.tsv.gz"
config = {"user":"root", 
          "password": "Cucumber#88", 
          "driver":"org.postgresql.Driver"}

In [21]:
#Write Review_df to RDS
Review_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
#Write Product_df to RDS
Product_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [ ]:
#Write Customer_df to RDS
Customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
#Write Vine_table_df to RDS
vine_table_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)